In [ ]:
using QuantumOptics
using NBInclude
using DataFrames
using LinearAlgebra
# lattice
@nbinclude("Lattice_Basis_Neighbors_Hops.ipynb"; regex=r"#.*executeme")
# mb tools
@nbinclude("QOJulia_utils.ipynb")
# chern 
@nbinclude("Chern_utils.ipynb")

LoadError: ArgumentError: Package DataFrames not found in current path.
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.

# Parameters

In [ ]:
param_dict=Dict{}()

In [ ]:
LL = [3,3]
N = LL[1]*LL[2]
vv_lattice = [ [1., 0], [0, 1.] ]
b1=[0, 0]
vv_basis = [b1]
param_dict["N"] = LL[1]*LL[2]

param_dict["LL"] = LL
param_dict["vv_lattice"] = vv_lattice
param_dict["vv_basis"] = vv_basis

nb = 1 # number of basis
t=-1 # hopping amplitude
p=1
q=3
ϕ=p/q # magnetic flux per plaquet
param_dict["gauge"] = "Landau"
param_dict["t"]=t
param_dict["ϕ"]=ϕ
θθ = [0, 0] # initial values for twist angles
param_dict["θθ"]=θθ

sites, site_coord, site_number = get_sites(LL, vv_lattice, vv_basis);
param_dict["sites"] = sites
param_dict["site_coord"] = site_coord
param_dict["site_number"] = site_number

pbc=true # periodic boundary condition
param_dict["pbc"] = pbc
neighbors = [ [1, 0, 1], [-1, 0, 1], [0, 1, 1], [0, -1, 1]  ]
neighbors_list = [ neighbors ]
param_dict["neighbors_list"] = neighbors_list

U = 0.1
param_dict["U"] = U;

LoadError: UndefVarError: get_sites not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [ ]:
# single ptl
basis = NLevelBasis(N) 
param_dict["basis"] = basis;

param_dict["model_H1"] = get_Hofstadter_hamiltonian

H_T = get_Hofstadter_hamiltonian(param_dict)
H1 = get_sp_op(basis, H_T)

E1, U1 = eigenstates(dense(H1));
E1

LoadError: UndefVarError: get_Hofstadter_hamiltonian not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [5]:
# projection
param_dict["sub_range"] = 1:9

sub_states = get_subset_states(H1, param_dict["sub_range"]);
proj1, P1, P1t = get_projector_op(basis, sub_states);
H1_proj = get_subspace_op(H1, P1, P1t);
H1_proj = (H1_proj+H1_proj')/2
param_dict["projection"] = proj1, P1, P1t;
#param_dict["projection"] = proj1, identityoperator(basis),identityoperator(basis);

LoadError: UndefVarError: get_subset_states not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


# Projected many body

In [6]:
param_dict["PN"] = [2];

LoadError: UndefVarError: param_dict not defined

In [7]:
basis_mb = get_basis_mb(param_dict)
param_dict["basis_mb"] = basis_mb;
param_dict["dimH_mb"] = length(basis_mb)

LoadError: UndefVarError: get_basis_mb not defined

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [4]:
H_NI_mb = get_mb_op(basis_mb, H1_proj);

LoadError: UndefVarError: get_mb_op not defined

In [5]:
U = param_dict["U"]
proj1, P1, P1t = param_dict["projection"]
P1 = P1.data
P1t = P1t.data
basis_mb = param_dict["basis_mb"]

LoadError: UndefVarError: param_dict not defined

In [40]:
using Einsum
@einsum U4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n];

In [41]:
basis_mb.basis_l = proj1

LoadError: type ManyBodyBasis has no field basis_l

In [42]:
Base.@propagate_inbounds function coefficient(occ_m, occ_n, at_indices, a_indices)
    any(==(0), (occ_m[m] for m in at_indices)) && return 0.
    any(==(0), (occ_n[n] for n in a_indices)) && return 0.
    C = prod(√, (occ_m[m] for m in at_indices)) * prod(√, (occ_n[n] for n in a_indices))
    for i in 1:length(occ_m)
        vm = occ_m[i]
        vn = occ_n[i]
        i in at_indices && (vm -= 1)
        i in a_indices && (vn -= 1)
        vm != vn && return zero(C)
    end
    return C
end

coefficient (generic function with 1 method)

In [43]:
function manybodyoperator_2(basis::ManyBodyBasis, op)
    N = length(basis)
    S = length(basis.onebodybasis)
    # @assert S^2 == length(op.basis_l)
    # @assert S^2 == length(op.basis_r)
    result = DenseOperator(basis)
    # op_data = reshape(op, S, S, S, S)
    occupations = basis.occupations
    @inbounds for m=1:N, n=1:N
        for l=1:S, k=1:S, j=1:S, i=1:S
            C = coefficient(occupations[m], occupations[n], (i, j), (k, l))
            result.data[m,n] += C*op[i, j, k, l]
        end
    end
    return result
end

manybodyoperator_2 (generic function with 1 method)

In [44]:
V_int_mb_2 = manybodyoperator_2(basis_mb, U4);

In [9]:
V_int_mb = get_Hub(param_dict)
V_int_mb = (V_int_mb+V_int_mb')/2;

LoadError: ArgumentError: The basis of the given operator has to either be equal to b or b ⊗ b where b is the 1st quantization basis associated to the nparticle basis.

In [45]:
H_mb = H_NI_mb + V_int_mb_2;
H_mb = (H_mb'+H_mb)/2

Operator(dim=45x45)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:9), states:45)
    -4.67662+0.0im         -0.059661-0.0461526im   …         0.0+0.0im
   -0.059661+0.0461526im    -4.80513+0.0im                   0.0+0.0im
  -0.0641126-0.0818293im  -0.0590447+0.0270451im             0.0+0.0im
 -0.00413404+0.0369159im    0.079025+0.012987im              0.0+0.0im
  -0.0245072-0.0902308im  -0.0102684+0.00435036im            0.0+0.0im
    0.059661-0.0461526im  -0.0416667-5.0e-17im     …         0.0+0.0im
     2.3e-16+2.0e-17im     0.0795691-1.0e-17im               0.0+0.0im
     0.03672-0.0619009im   -0.032775+0.0117393im             0.0+0.0im
  -0.0309159+0.0186229im    0.118918+0.143926im       -0.0309159-0.0186229im
         0.0+0.0im         -0.059661+0.0461526im             0.0+0.0im
         0.0+0.0im         -0.106904-0.0773571im   …         0.0+0.0im
         0.0+0.0im        -0.0762432-0.0644885im             0.0+0.0im
         0.0+0.0im         0.0949786

In [46]:
EE, UU = eigenstates(dense(H_mb));

In [47]:
EE

45-element Vector{Float64}:
 -5.054811414238365
 -4.922258372106864
 -4.762617623167232
 -4.648220715332506
 -4.636587991712217
 -4.559035023929745
 -1.5383739317009386
 -1.4467467481735872
 -1.4254623188198758
 -1.4106582232449938
 -1.3305774301104387
 -1.3049913354484097
 -1.2591914063909062
  ⋮
  3.5920381722607457
  3.700600626140661
  3.8268578014385013
  3.9702381768409483
  3.974805203786205
  4.002413258701591
  4.709189433471977
  4.728151846404004
  4.77950561123288
  4.8585517061837065
  4.883355609490756
  5.027900828705339

In [364]:
V_int_mb_2 = get_Hub_2(param_dict)
V_int_mb_2 = (V_int_mb_2+V_int_mb_2')/2;

In [368]:
H_mb = H_NI_mb + V_int_mb_2;

In [369]:
EE, UU = eigenstates(dense(H_mb));

In [370]:
EE

6-element Vector{Float64}:
 -5.518456494918703
 -5.518080068690142
 -5.516163693965257
 -5.511081348947812
 -5.510893135833533
 -5.509934948471088